In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

data = Planetoid(root="data/Cora", name="Cora")[0]

class GCN(torch.nn.Module):
    def __init__(self, in_ch, hid=16, out_ch=7):
        super().__init__()
        self.c1 = GCNConv(in_ch, hid)
        self.c2 = GCNConv(hid, out_ch)
    def forward(self, x, edge_index):
        x = F.relu(self.c1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        return self.c2(x, edge_index)

m = GCN(data.num_features, out_ch=int(data.y.max())+1)
opt = torch.optim.Adam(m.parameters(), lr=0.01, weight_decay=5e-4)

for _ in range(200):
    m.train(); opt.zero_grad()
    out = m(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward(); opt.step()

m.eval()
pred = out.argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean().item()
print("test_acc =", acc)


Processing...
Done!


test_acc = 0.7509999871253967
